# Assignment

Instructions:

This assignment requires you to implement multi Al agent system using generative Al, with multiple capabilities.

Objective:

Create a design and develop multiple Al agents with distinct roles, enabling them to collaborate, communicate, and perform complex tasks autonomously. Each agent leverages its specialized capabilities to solve problems or automate processes.

Task:

1. Create an Al agent to serve as customer service, recommendation of product or service, calculate forecasting and generate reports for giving insight to the user.
2. Combine the Al agent so that we have an Al agent system.

Questions and Coding Tasks:

1. Generative Model Setup
Implement a text-based generative model using a large language model (LLM) that takes user input and generates a relevant and coherent response based on the user"s query or request.

Coding Task:

Write the code to be chatbot application that has multiple capabilities

# Assignment Chat Bot: MarketingAIAssistant+

The AI Assistant will help in:
1. What is the current market trend in fashion using Google Search. (**Market Researcher**)
2. Query product stok in database. (**Data Analyst**)
3. Forecast demand. (**Data Scientist**)
4. Polish answer (**Customer Support**)

# Install and import libraries

In [ ]:
!pip install -q crewai
!pip install -q "crewai[tools]"
!pip install -q langchain_community
!pip install -q openai
!pip install -q serper
!pip install -q scipy
!pip install -q scikit-learn
!pip install -q lightgbm
!pip install -q langchain_openai
!pip install -q faker

ERROR: Could not find a version that satisfies the requirement serper (from versions: none)
ERROR: No matching distribution found for serper


In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Agentic AI related
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool, SerperDevTool, ScrapeWebsiteTool, CSVSearchTool

from langchain_openai import ChatOpenAI

import os

from IPython.display import Markdown

# Forecast Machine Learning
import ast
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random
from IPython.display import Markdown

In [ ]:
from google.colab import userdata

os.environ["OPENAI_API_KEY"]= userdata.get("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

# Dataset

In [ ]:
fashion_products = [
    "Crimson Velvet Midi Dress (Women)",
    "Royal Blue Wool Coat (Men)",
    "Emerald Green Silk Scarf (Unisex)",
    "Sunset Orange Maxi Skirt (Women)",
    "Midnight Black Leather Jacket (Men)",
    "Ivory White Knit Sweater (Women)",
    "Cherry Red Sneakers (Men)",
    "Lavender Mist Blouse (Women)",
    "Cobalt Blue Running Shoes (Men)",
    "Pearl Grey Cashmere Cardigan (Women)",
    "Jet Black Bomber Jacket (Men)",
    "Blush Pink Flared Pants (Women)",
    "Sapphire Blue Button-Up Shirt (Men)",
    "Golden Yellow Wool Hat (Unisex)",
    "Charcoal Grey Hoodie (Women)",
    "Maroon Pleated Skirt (Women)",
    "Deep Sea Blue Pullover (Men)",
    "Olive Green Utility Jacket (Unisex)",
    "Slate Grey Turtleneck (Women)",
    "Burnt Copper Chinos (Men)"
]

In [ ]:
product_list = []
dataset = []
total_products = len(fashion_products)

# Create product-price dict
for i in range(total_products):
    product_id = f"PROD{1000+i}"
    product_name = fashion_products[i]
    price = random.randint(50000, 500000)
    product_list.append([product_id, product_name, price])

# Create dummy dataset
week_year_index = -1
for w in range(1, 53, 1):
  week = w
  for y in [2021, 2022, 2023, 2024]:
    year = y
    week_year_index += 1
    for p in product_list:
      product_id = p[0]
      product_name = p[1]
      stock = random.randint(0, 100)
      promotion = 1 if random.random() <= 0.25 else 0
      discount = random.choice(np.linspace(-0.7, 0, 8)) if promotion==1 else random.choice([0, 0.1])
      price = p[2]
      qty_sold = random.randint(1, 20) if promotion==0 else random.choice(np.arange(4, 40, 4))

      dataset.append([week_year_index, week, year, product_id, product_name, stock, promotion, discount, price, qty_sold])

In [ ]:
df = pd.DataFrame(dataset, columns = ["week_year_index", "week", "year", "product_id", "product_name", "stock", "promotion", "discount", "price", "qty_sold"])

df.head()

,week_year_index,week,year,product_id,product_name,stock,promotion,discount,price,qty_sold
0,0,1,2021,PROD1000,Crimson Velvet Midi Dress (Women),72,1,0.0,177899,8
1,0,1,2021,PROD1001,Royal Blue Wool Coat (Men),5,0,0.1,189419,2
2,0,1,2021,PROD1002,Emerald Green Silk Scarf (Unisex),95,0,0.1,352052,8
3,0,1,2021,PROD1003,Sunset Orange Maxi Skirt (Women),9,0,0.0,150491,7
4,0,1,2021,PROD1004,Midnight Black Leather Jacket (Men),91,1,0.0,399589,36


# Build Multi-Agent System

## Tools

### Pre-built

In [ ]:
# Pre-built tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

### Current Stock Tool

In [ ]:
current_df = df[df["week_year_index"]==df["week_year_index"].max()][["product_id", "product_name", "stock"]]
for i in current_df.columns:
  current_df[i] = current_df[i].astype(str)

current_df.to_csv("current_stock.csv", index=False)

stock_tool = CSVSearchTool(csv='/content/current_stock.csv')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.13it/s]


## AI Agents

In [ ]:
market_research_agent = Agent(
    role="Trend Research Agent",
    goal="Retrieve relevant data on fashion",
    backstory=(
      "Working on retail fashion store company, this agent responsible for gathering "
      "relevant information on key trends in the fashion industry for 2024 and beyond."
    ),
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    tools = [search_tool, scrape_tool],
    max_iter = 2,
    allow_delegation=True,
    verbose=True
)

stock_manager_agent = Agent(
    role="Stock Manager",
    goal="Retrieve and analyze internal company stock data",
    backstory=(
      "Working on retail fashion store company, this agent is able to obtain company current stock data, "
      "to provide insights and strategy for inventory planning to avoid stock-outs. "
    ),
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    tools = [stock_tool],
    max_iter = 2,
    allow_delegation=True,
    verbose=True
)

data_scientist_agent = Agent(
    role="Data scientist",
    goal="Forecast product demand to provide insights for inventory planning strategy to avoid stockouts",
    backstory=(
      "Working on retail fashion store company, this agent uses mathematical, statistical and "
      "machine learing model to turn company stock data and market trend into "
      "company inventory planning strategy."
    ),
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    tools = [stock_tool, search_tool, scrape_tool],
    max_iter = 2,
    allow_delegation=True,
    verbose=True
)

customer_support_agent = Agent(
    role="Customer Support Representative",
    goal="Assist customers with inquiries and communicate with clear, easy to understand and professional tone.",
    backstory=(
        "Working on retail fashion store company, this agent communicate with user to provide insightful "
        "conversation and company strategy regarding product promotion and inventory planning. This agent "
        "user are internal senior staff and manager alike"
    ),
    allow_delegation=False,
    verbose=True
)

### Creating Tasks

In [ ]:
query_stock_task = Task(
    description=(
        "Retrieve current stock level from internal company data based on user inquiry. "
        "User name={user} "
        "Inquiry={inquiry}"
        ),
    expected_output="Information about current stock level of each product",
    agent=stock_manager_agent,
)

trend_research_task = Task(
    description=(
        "Research emerging trends in 2024 and beyond on fashion industry based on user inquiry."
        "User name={user}  "
        "Inquiry={inquiry}"
        ),
    expected_output="All the information organized with key trends and insights",
    agent=market_research_agent,
)

forecast_task = Task(
    description=(
        "Forecast product demand to provide insights for inventory planning strategy to avoid stockouts. "
        "Infer the product from user inquiry ."
        "User name={user} "
        "Inquiry={inquiry}"
    ),
    expected_output="All the information organized with key trends and insights",
    agent=data_scientist_agent,
)

customer_support_task = Task(
    description=(
        "Assist customers with inquiries regarding product availability, trend, forecast and inventory strategy. "
        "Provide helpful, accurate, and professional responses."
        "User name={user} "
        "Inquiry={inquiry}"
    ),
    expected_output=(
        "A professional response to the customer's inquiry. Ensure all parts of the question are answered."
    ),
    agent=customer_support_agent,
)

### Creating the Crew

In [ ]:
crew = Crew(
    agents=[market_research_agent, stock_manager_agent, data_scientist_agent, customer_support_agent],
    tasks=[query_stock_task, trend_research_task, forecast_task, customer_support_task],
    verbose=True,
    memory=True
)

### Running the Crew

In [ ]:
class FrontDesk():
  def __init__(self):
    self.crew = crew
    self.user = None
    self.inquiry = None
    self.memory = []
    self.result = None

  def kickoff(self):
    self.user = input("Please enter your name: ")
    self.inquiry = input("Please enter your inquiry: ")
    self.result = crew.kickoff(inputs={"user":self.user,"inquiry":self.inquiry})
    self.memory.append({"user":self.user,"inquiry":self.inquiry})
    return self.result

  def response(self):
    self.kickoff()
    return Markdown(self.result.raw)

In [ ]:
front_desk = FrontDesk()

front_desk.response()

Please enter your name: Biyan
Please enter your inquiry: Recommend me jacket from your store please
# Agent: Stock Manager
## Task: Retrieve current stock level from internal company data based on user inquiry. User name=Biyan Inquiry=Recommend me jacket from your store please


# Agent: Stock Manager
## Final Answer: 
Current stock level of jackets for men in the store:

1. Product ID: PROD1010
   Product Name: Jet Black Bomber Jacket (Men)
   Stock Level: 76

2. Product ID: PROD1004
   Product Name: Midnight Black Leather Jacket (Men)
   Stock Level: 15

3. Product ID: PROD1017
   Product Name: Olive Green Utility Jacket (Unisex)
   Stock Level: 44

There are three jackets available for men in the store: Jet Black Bomber Jacket, Midnight Black Leather Jacket, and Olive Green Utility Jacket.


# Agent: Trend Research Agent
## Task: Research emerging trends in 2024 and beyond on fashion industry based on user inquiry.User name=Biyan  Inquiry=Recommend me jacket from your store please



Thank you for your inquiry, Biyan. Yes, we have jackets for men available in our store. Currently, we stock the following options:

1. **Product ID:** PROD1010  
   **Product Name:** Jet Black Bomber Jacket (Men)  
   **Stock Level:** 76

2. **Product ID:** PROD1004  
   **Product Name:** Midnight Black Leather Jacket (Men)  
   **Stock Level:** 15

3. **Product ID:** PROD1017  
   **Product Name:** Olive Green Utility Jacket (Unisex)  
   **Stock Level:** 44

In total, we have three styles of jackets that cater to men's fashion needs. If you're looking for a recommendation, based on the current stock levels and the emerging trends in men's jackets for 2024, I would recommend the Jet Black Bomber Jacket. Bomber jackets are a classic style that is versatile and can be easily paired with different outfits. Additionally, bomber jackets are predicted to be popular in 2024, aligning with the latest trends in men's fashion. 

Please let me know if you'd like more information or assistance with your jacket selection!

In [ ]:
front_desk.response()

Please enter your name: Yunti
Please enter your inquiry: Latest trend in women fashion in Indonesia
# Agent: Stock Manager
## Task: Retrieve current stock level from internal company data based on user inquiry. User name=Yunti Inquiry=Latest trend in women fashion in Indonesia


# Agent: Stock Manager
## Thought: I need to search the internal company data to retrieve the current stock level of each product mentioned in the inquiry.
## Using tool: Search a CSV's content
## Tool Input: 
"{\"search_query\": \"Jet Black Bomber Jacket\"}"
## Tool Output: 
Relevant Content:
product_id: PROD1010, product_name: Jet Black Bomber Jacket (Men), stock: 76

product_id: PROD1004, product_name: Midnight Black Leather Jacket (Men), stock: 15

product_id: PROD1017, product_name: Olive Green Utility Jacket (Unisex), stock: 44


# Agent: Stock Manager
## Final Answer: 
Current stock level of jackets for men in the store:

1. Product ID: PROD1010
   Product Name: Jet Black Bomber Jacket (Men)
   Stock Lev

Thank you for your inquiry, Yunti! In Indonesia for the year 2024, the latest trends in women's fashion showcase a vibrant mix of tradition, innovation, and cultural diversity. There is a surge in demand for modest fashion styles, reflecting the country's cultural and religious diversity. Events like Plaza Indonesia Fashion Week 2024 and Bali Fashion Trend 2024 highlight a blend of contemporary and traditional styles, combining modern youth fashion with cultural elements.

Indonesian designers are reviving traditional fabrics to create unique and creative fashion pieces, offering a blend of heritage and innovation in the fashion industry. Additionally, women in the textile industry in Indonesia are advocating for more rights and recognition for their contributions to the booming textile sector.

These trends emphasize the importance of cultural heritage, creativity, and inclusivity in Indonesian fashion for 2024. Designers are exploring traditional fabrics and colors while incorporating modern silhouettes and styles to create a unique and diverse fashion landscape. The fashion industry in Indonesia is evolving to celebrate the country's rich cultural heritage while embracing innovation and creativity.

If you would like more detailed insights or assistance with inventory planning based on these trends, please feel free to reach out. Thank you for considering our store for your fashion needs!

In [ ]:
front_desk.response()

Please enter your name: Biyan
Please enter your inquiry: Provide inventory planning for men's jacket based on the latest trend in Indonesia
# Agent: Stock Manager
## Task: Retrieve current stock level from internal company data based on user inquiry. User name=Biyan Inquiry=Provide inventory planning for men's jacket based on the latest trend in Indonesia
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FixedCSVSearchToolSchema
search_query
  Input should be a valid string [type=string_type, input_value={'description': "Men's jackets stock level"}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search a CSV's content accepts these inputs: Tool Name: Search a CSV's content
Tool Arguments: {'search_query': {'description': "Mandatory search query you want to use to search the CSV's content", 'type': 'str'}}
Tool Description: A tool that can be used to se

Thank you for your inquiry, Biyan!

Regarding inventory planning for men's jackets based on the latest trends in Indonesia for 2024, the Coats & Jackets market is projected to generate a revenue of approximately US$1.12 billion, with an annual growth rate of 4.11% from 2024 to 2029. This growth reflects a rising demand for jackets, specifically lightweight and waterproof options that cater to the tropical climate of Indonesia. With an anticipated volume reaching 18.5 million units by 2029, it’s crucial to align our inventory strategy with these trends.

Current fashion trends indicate that practical and versatile outerwear options are becoming increasingly popular among consumers. In particular, we are seeing a demand for styles such as utility jackets, packable puffers, and classic bombers that offer both style and functionality.

To develop an effective inventory planning strategy for men's jackets, I suggest the following:

1. **Stock Assessment**: Review current stock levels of men's jackets in our inventory. For example, a popular item is the Jet Black Bomber Jacket (Product ID: PROD1010), with a current stock of 76 units, which aligns perfectly with the trend towards versatile outerwear.

2. **Forecast Demand**: Given the market trends and growth projections, we should anticipate a potential increase in sales for jackets that meet the described needs. It may be beneficial to predict a sales increase of around 10-20% for popular styles like waterproof jackets and bombers during peak shopping seasons.

3. **Inventory Adjustment**:
   - **Increase Stock Levels**: Consider increasing the stock of best-selling items such as the Jet Black Bomber Jacket from 76 to at least 90 units in anticipation of demand surges.
   - **Diversity In Styles**: Introduce additional styles such as lightweight rain jackets and insulated coats to diversify our inventory and meet customer preferences.

4. **Monitoring Trends**: Continuously analyze sales patterns and customer feedback to refine our inventory strategy. Keeping track of trends in social media and fashion reports can provide valuable insights to stay ahead of consumer demands.

5. **Marketing Approach**: Promote the trending jackets through targeted marketing campaigns highlighting their practicality and aligning with current fashion trends to stimulate interest and drive sales.

By aligning our inventory strategy with marketplace trends and focusing on versatile product offerings, we can enhance customer satisfaction, optimize sales opportunities, and maintain a strong competitive position in the Indonesian men's outerwear market. Thank you for considering our retail insights, and please feel free to reach out for any additional information or support!